In [ ]:
# 每个点分为上左下右，左右下与超级节点连，从最后一排往上统计有效储水区域，此时会包含密闭空间，需要减去。
from typing import List


class Solution:

    def reservoir(self, shape: List[str]) -> int:
        rows, cols = len(shape), len(shape[0])
        u = [[0] * cols for _ in range(rows)]
        l = [[0] * cols for _ in range(rows)]
        d = [[0] * cols for _ in range(rows)]
        r = [[0] * cols for _ in range(rows)]
        idx = 0
        for i in range(rows):
            for j in range(cols):
                u[i][j] = idx
                idx += 1
                l[i][j] = idx
                idx += 1
                d[i][j] = idx
                idx += 1
                r[i][j] = idx
                idx += 1
        n = rows * cols * 4
        roots = list(range(n + 1))  # 加上超级节点n

        def find(x):
            if roots[x] != x:
                roots[x] = find(roots[x])
            return roots[x]

        def union(x, y):
            rx, ry = find(x), find(y)
            if rx != ry:
                roots[rx] = ry

        ans = 0
        # 左右下三方与超级节点相连
        for i in range(rows):
            union(l[i][0], n)
            union(r[i][cols - 1], n)
        for j in range(cols):
            union(d[rows - 1][j], n)
        # 从下往上逐层 统计ans:能容纳水的有效空间（包含密闭空间）
        for i in range(rows - 1, -1, -1):
            # 与下连
            if i != rows - 1:
                for j in range(cols):
                    union(d[i][j], u[i + 1][j])
            # 与右连
            for j in range(cols - 1):
                union(r[i][j], l[i][j + 1])
            # 内部连
            for j in range(cols):
                x = shape[i][j]
                if x == 'l':
                    union(l[i][j], d[i][j])
                    union(u[i][j], r[i][j])
                elif x == 'r':
                    union(l[i][j], u[i][j])
                    union(d[i][j], r[i][j])
                else:
                    union(l[i][j], u[i][j])
                    union(l[i][j], r[i][j])
                    union(l[i][j], d[i][j])
            # 检查不与超级节点相连的个数
            for x in range(u[i][0], r[i][cols - 1] + 1):
                ans += find(x) != find(n)
            # print(i, ans)
        # ans 减去 密闭空间
        # 把第一行与超级节点相连 为了方便统计密闭空间
        for j in range(cols):
            union(u[0][j], n)
        for x in range(n):
            ans -= find(x) != find(n)
        return ans // 2
